<a href="https://colab.research.google.com/github/goudapranay/Climate-Adjusted-Spatial-Prioritization-of-Rice-Fallow-Intensification/blob/main/SOC_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==========================================================
# Climate-Smart Agriculture in India's Rice-Fallow Belt
# FINAL INTEGRATED VERSION
# Raster Mask + Climate + Weighted CSA + Priority + Quadrants
# ==========================================================

import ee
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
from shapely.geometry import shape
from matplotlib.patches import Patch

# ==========================================================
# CONFIGURATION
# ==========================================================

CONFIG = {
    "project_id": "ee-goudapranay",
    "districts_asset": "projects/ee-goudapranay/assets/rice_fallows_ha_india",
    "raster_mask": "projects/ee-goudapranay/assets/rice_fallow_mask",
    "output_dir": "csa_india_final",
    "gcm_model": "GFDL-CM3",
    "scenario": "rcp85",
    "period": ("2081-01-01", "2100-12-31"),

    "soc_params": {
        "bulk_density": 1.3,
        "depth_m": 0.3
    },

    "carbon_params": {
        "AGB": 3.5,
        "CF": 0.45,
        "R": 0.24,
        "CR": 0.1,
        "SOC_increment": 0.4
    },

    "climate_sensitivity": {
        "temp_coeff": -0.04,
        "precip_coeff": 0.015,
        "temp_floor": 0.6
    },

    "economic": {
        "carbon_price_usd": 10,
        "pulse_yield_t_ha": 1.0,
        "pulse_price_usd_t": 280,
        "seed_cost_usd_ha": 20
    },

    "min_fallow_area": 10000,  # ✅ Updated to 10,000 ha

    "csa_weights": {
        "mitigation": 0.5,
        "adaptation": 0.5
    },

    "priority_weights": {
        "performance": 0.6,
        "area": 0.4
    }
}

# ==========================================================
# INITIALIZE EARTH ENGINE
# ==========================================================

def initialize_ee():
    try:
        ee.Initialize(project=CONFIG["project_id"])
    except:
        ee.Authenticate()
        ee.Initialize(project=CONFIG["project_id"])

# ==========================================================
# LOAD DISTRICTS
# ==========================================================

def load_districts():
    districts_fc = ee.FeatureCollection(CONFIG["districts_asset"])
    info = districts_fc.getInfo()

    rows, geoms = [], []
    for f in info["features"]:
        geom = f.get("geometry")
        if not geom:
            continue
        shapely_geom = shape(geom)
        if shapely_geom.is_empty:
            continue
        rows.append({"District": f["properties"]["District"]})
        geoms.append(shapely_geom)

    df = pd.DataFrame(rows)
    gdf = gpd.GeoDataFrame(df, geometry=geoms, crs="EPSG:4326")

    return df, gdf, districts_fc

# ==========================================================
# COMPUTE MASKED STATS
# ==========================================================

def compute_masked_stats(districts_fc):

    fallow_mask = ee.Image(CONFIG["raster_mask"]).unmask(0)
    pixel_area_ha = ee.Image.pixelArea().divide(10000)
    fallow_area_img = fallow_mask.multiply(pixel_area_ha)

    # SOC
    soc_raw = ee.Image(
        "OpenLandMap/SOL/SOL_ORGANIC-CARBON_USDA-6A1C_M/v02"
    ).select("b0")

    soc_gkg = soc_raw.multiply(5)
    BD = CONFIG["soc_params"]["bulk_density"]
    depth = CONFIG["soc_params"]["depth_m"]

    soc_stock = soc_gkg.multiply(BD).multiply(depth).multiply(10)
    soc_masked = soc_stock.updateMask(fallow_mask)

    # Climate
    ic = (
        ee.ImageCollection("NASA/NEX-GDDP")
        .filter(ee.Filter.eq("model", CONFIG["gcm_model"]))
        .filter(ee.Filter.eq("scenario", CONFIG["scenario"]))
        .filterDate(*CONFIG["period"])
    )

    tmin = ic.select("tasmin").mean()
    tmax = ic.select("tasmax").mean()
    temp_mean = tmin.add(tmax).divide(2)

    pr_year = ic.select("pr").mean().multiply(365.25 * 24 * 3600)

    temp_masked = temp_mean.updateMask(fallow_mask)
    pr_masked = pr_year.updateMask(fallow_mask)

    pr_stack = ic.select("pr").map(lambda img: img.multiply(365.25 * 24 * 3600))
    temp_cv = ic.select("tasmax").reduce(ee.Reducer.stdDev()) \
        .divide(ic.select("tasmax").mean())
    pr_cv = pr_stack.reduce(ee.Reducer.stdDev()).divide(pr_stack.mean())

    temp_cv_masked = temp_cv.updateMask(fallow_mask)
    pr_cv_masked = pr_cv.updateMask(fallow_mask)

    def zonal(img, reducer, scale):
        return img.reduceRegions(
            collection=districts_fc,  # ❌ TYPO HERE
            reducer=reducer,
            scale=scale
        )

    climate_scale = 25000

    area_stats = zonal(fallow_area_img, ee.Reducer.sum(), 250)
    soc_stats = zonal(soc_masked, ee.Reducer.mean(), 250)
    temp_stats = zonal(temp_masked, ee.Reducer.mean(), climate_scale)
    pr_stats = zonal(pr_masked, ee.Reducer.mean(), climate_scale)
    temp_cv_stats = zonal(temp_cv_masked, ee.Reducer.mean(), climate_scale)
    pr_cv_stats = zonal(pr_cv_masked, ee.Reducer.mean(), climate_scale)

    def to_dict(stats, key):
        return {
            f["properties"]["District"]:
                f["properties"].get(key, np.nan)
            for f in stats.getInfo()["features"]
        }

    return {
        "Rice_Fallows": to_dict(area_stats, "sum"),
        "Baseline_SOC": to_dict(soc_stats, "mean"),
        "Future_Temp": to_dict(temp_stats, "mean"),
        "Future_Precip": to_dict(pr_stats, "mean"),
        "Temp_CV": to_dict(temp_cv_stats, "mean"),
        "Precip_CV": to_dict(pr_cv_stats, "mean")
    }

# ==========================================================
# GENERATE 8 PUBLICATION FIGURES + QUADRANT PLOT
# ==========================================================

# ==========================================================
# GENERATE 8 PUBLICATION FIGURES (ALL DISTRICTS + ELIGIBLE HIGHLIGHTED)
# ==========================================================

def generate_figures(df, gdf, output_dir):
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)

    # Create full GeoDataFrame (all districts)
    gdf_full = gpd.GeoDataFrame(df, geometry=gdf.geometry, crs="EPSG:4326")

    # Create eligibility mask
    eligible = df["compute_flag"]

    def add_north_arrow(ax, x=0.05, y=0.95, length=0.05, color="black"):
        ax.annotate('', xy=(x, y), xytext=(x, y - length),
                    xycoords='axes fraction', textcoords='axes fraction',
                    arrowprops=dict(arrowstyle='->', lw=3, color=color))
        ax.text(x, y + 0.01, 'N', transform=ax.transAxes,
                fontsize=24, fontweight='bold', ha='center', va='bottom', color=color)

    figsize = (30, 20)
    linewidth = 0.3
    edgecolor = "black"
    missing_color = "lightgrey"

    # Fig 1: Rice-Fallow Distribution (All districts)
    fig, ax = plt.subplots(1, figsize=figsize)
    gdf_full.plot(column="Rice_Fallows", cmap="OrRd", linewidth=linewidth, ax=ax, edgecolor=edgecolor,
                  legend=True, missing_kwds={"color": missing_color})
    add_north_arrow(ax)
    ax.set_title("Fig 1. Spatial Distribution of Rice-Fallow Area Across India",
                 fontdict={"fontsize": 26, "fontweight": "bold"}, pad=20)
    ax.axis("off")
    cax = fig.axes[-1]
    cax.set_ylabel("Area (ha)", fontsize=20, fontweight="bold")
    cax.tick_params(labelsize=16)
    plt.savefig(output_dir / "Fig1_RiceFallow_India.png", bbox_inches="tight", dpi=300, facecolor="white")
    plt.close()

    # Fig 2: Top 20 Districts (Bar Chart - Only eligible)
    top20 = df[eligible].nlargest(20, "Rice_Fallows")
    fig, ax = plt.subplots(figsize=(16, 8))
    ax.barh(top20["District"], top20["Rice_Fallows"], color="maroon")
    ax.set_xlabel("Rice-Fallow Area (ha)", fontsize=14)
    ax.set_ylabel("District", fontsize=14)
    ax.set_title("Fig 2. Top 20 Districts by Rice-Fallow Area", fontsize=16, fontweight="bold")
    plt.tight_layout()
    plt.savefig(output_dir / "Fig2_Top20_RiceFallow.png", dpi=300)
    plt.close()

    # Fig 3: Carbon Sequestration (Eligible colored, others grey)
    fig, ax = plt.subplots(1, figsize=figsize)
    gdf_full[~eligible].plot(color=missing_color, linewidth=linewidth, ax=ax, edgecolor=edgecolor)
    gdf_full[eligible].plot(column="Mitigation_CO2", cmap="YlGn", linewidth=linewidth, ax=ax, edgecolor=edgecolor,
                           legend=True)
    add_north_arrow(ax)
    ax.set_title("Fig 3. Climate-Adjusted Carbon Sequestration Potential (RCP8.5)",
                 fontdict={"fontsize": 26, "fontweight": "bold"}, pad=20)
    ax.axis("off")
    cax = fig.axes[-1]
    cax.set_ylabel("CO₂ (tons/year)", fontsize=20, fontweight="bold")
    cax.tick_params(labelsize=16)
    plt.savefig(output_dir / "Fig3_Carbon_Sequestration.png", bbox_inches="tight", dpi=300, facecolor="white")
    plt.close()

    # Fig 4: Yield Stability (Eligible colored, others grey)
    fig, ax = plt.subplots(1, figsize=figsize)
    gdf_full[~eligible].plot(color=missing_color, linewidth=linewidth, ax=ax, edgecolor=edgecolor)
    gdf_full[eligible].plot(column="Adaptation_Stability", cmap="RdYlGn", linewidth=linewidth, ax=ax, edgecolor=edgecolor,
                           legend=True)
    add_north_arrow(ax)
    ax.set_title("Fig 4. Projected Yield Stability Under Climate Change (RCP8.5)",
                 fontdict={"fontsize": 26, "fontweight": "bold"}, pad=20)
    ax.axis("off")
    cax = fig.axes[-1]
    cax.set_ylabel("Stability Index (0–1)", fontsize=20, fontweight="bold")
    cax.tick_params(labelsize=16)
    plt.savefig(output_dir / "Fig4_Yield_Stability.png", bbox_inches="tight", dpi=300, facecolor="white")
    plt.close()

    # Fig 5: Baseline SOC (Eligible colored, others grey)
    fig, ax = plt.subplots(1, figsize=figsize)
    gdf_full[~eligible].plot(color=missing_color, linewidth=linewidth, ax=ax, edgecolor=edgecolor)
    gdf_full[eligible].plot(column="Baseline_SOC", cmap="YlOrBr", linewidth=linewidth, ax=ax, edgecolor=edgecolor,
                           legend=True)
    add_north_arrow(ax)
    ax.set_title("Fig 5. Baseline Soil Organic Carbon Stock in Rice-Fallow Districts (>10,000 ha)",
                 fontdict={"fontsize": 26, "fontweight": "bold"}, pad=20)
    ax.axis("off")
    cax = fig.axes[-1]
    cax.set_ylabel("SOC (t C/ha)", fontsize=20, fontweight="bold")
    cax.tick_params(labelsize=16)
    plt.savefig(output_dir / "Fig5_Baseline_SOC.png", bbox_inches="tight", dpi=300, facecolor="white")
    plt.close()

    # Fig 6: CSA Index (Eligible colored, others grey)
    fig, ax = plt.subplots(1, figsize=figsize)
    gdf_full[~eligible].plot(color=missing_color, linewidth=linewidth, ax=ax, edgecolor=edgecolor)
    gdf_full[eligible].plot(column="CSA_Index", cmap="viridis", linewidth=linewidth, ax=ax, edgecolor=edgecolor,
                           legend=True)
    add_north_arrow(ax)
    ax.set_title("Fig 6. Climate-Smart Agriculture (CSA) Performance Index",
                 fontdict={"fontsize": 26, "fontweight": "bold"}, pad=20)
    ax.axis("off")
    cax = fig.axes[-1]
    cax.set_ylabel("CSA Index (0–1)", fontsize=20, fontweight="bold")
    cax.tick_params(labelsize=16)
    plt.savefig(output_dir / "Fig6_CSA_Index.png", bbox_inches="tight", dpi=300, facecolor="white")
    plt.close()

    # Fig 7: Net Benefit per Hectare (Eligible colored, others grey)
    fig, ax = plt.subplots(1, figsize=figsize)
    gdf_full[~eligible].plot(color=missing_color, linewidth=linewidth, ax=ax, edgecolor=edgecolor)
    gdf_full[eligible].plot(column="Net_Benefit_per_Ha", cmap="Purples", linewidth=linewidth, ax=ax, edgecolor=edgecolor,
                           legend=True)
    add_north_arrow(ax)
    ax.set_title("Fig 7. Net Annual Benefit per Hectare for Farmers (USD/ha)",
                 fontdict={"fontsize": 26, "fontweight": "bold"}, pad=20)
    ax.axis("off")
    cax = fig.axes[-1]
    cax.set_ylabel("USD/ha", fontsize=20, fontweight="bold")
    cax.tick_params(labelsize=16)
    plt.savefig(output_dir / "Fig7_Net_Benefit_per_Ha.png", bbox_inches="tight", dpi=300, facecolor="white")
    plt.close()

    # Fig 8: Priority Classification WITH LEGEND (Eligible colored, others grey)
    priority_colors = {
        "High Performance – High Scale": "darkgreen",
        "High Performance – Low Scale": "forestgreen",
        "Low Performance – High Scale": "gold",
        "Low Performance – Low Scale": "lightcoral",
        "Not Eligible": "lightgrey"
    }

    fig, ax = plt.subplots(1, figsize=figsize)
    # Plot non-eligible districts in grey
    gdf_full[~eligible].plot(color="lightgrey", linewidth=linewidth, ax=ax, edgecolor=edgecolor)
    # Plot eligible districts with priority colors
    eligible_gdf = gdf_full[eligible]
    eligible_gdf.plot(color=eligible_gdf["Priority_Class"].map(priority_colors),
                     linewidth=linewidth, ax=ax, edgecolor=edgecolor)

    # Add custom legend
    legend_elements = [
        Patch(facecolor="darkgreen", label="High Performance – High Scale"),
        Patch(facecolor="forestgreen", label="High Performance – Low Scale"),
        Patch(facecolor="gold", label="Low Performance – High Scale"),
        Patch(facecolor="lightcoral", label="Low Performance – Low Scale"),
        Patch(facecolor="lightgrey", label="Not Eligible (<10,000 ha)")
    ]
    ax.legend(handles=legend_elements, loc="lower left", fontsize=14, frameon=True)

    add_north_arrow(ax)
    ax.set_title("Fig 8. Priority Classification of Rice-Fallow Districts",
                 fontdict={"fontsize": 26, "fontweight": "bold"}, pad=20)
    ax.axis("off")
    plt.savefig(output_dir / "Fig8_Priority_Classification.png", bbox_inches="tight", dpi=300, facecolor="white")
    plt.close()

    print(f"✅ All 8 figures saved to {output_dir}")

# ==========================================================
# MAIN
# ==========================================================

def main():

    initialize_ee()

    output_dir = Path(CONFIG["output_dir"])
    output_dir.mkdir(exist_ok=True)

    df, gdf, districts_fc = load_districts()
    stats = compute_masked_stats(districts_fc)

    for col, dct in stats.items():
        df[col] = df["District"].map(dct)

    df["Rice_Fallows"] = df["Rice_Fallows"].fillna(0)
    df["compute_flag"] = df["Rice_Fallows"] > CONFIG["min_fallow_area"]

    # Initialize result columns
    result_cols = [
        "Mitigation_CO2",
        "Adaptation_Stability",
        "CSA_Index",
        "Area_Norm",
        "Priority_Index",
        "Priority_Class",
        "Net_Annual_Benefit",
        "Net_Benefit_per_Ha"
    ]
    for col in result_cols:
        df[col] = np.nan

    # ---------------- Core Computation ----------------

    for idx in df[df["compute_flag"]].index:

        row = df.loc[idx]
        area = row["Rice_Fallows"]
        if area <= 0:
            continue

        # Climate Productivity Index
        sens = CONFIG["climate_sensitivity"]

        temp_effect = 1 + sens["temp_coeff"] * (row["Future_Temp"] - 28)
        temp_effect = max(temp_effect, sens["temp_floor"])

        precip_effect = 1 + sens["precip_coeff"] * \
            ((row["Future_Precip"] - 1200) / 1200)

        CPI = temp_effect * precip_effect

        # Carbon
        cp = CONFIG["carbon_params"]

        AGB_climate = cp["AGB"] * CPI
        SOC_inc_climate = cp["SOC_increment"] * CPI

        C_ab = AGB_climate * cp["CF"] * (1 - cp["CR"])
        C_bb = AGB_climate * cp["R"] * cp["CF"]

        C_total = C_ab + C_bb + SOC_inc_climate
        co2 = area * C_total * (44 / 12)

        df.at[idx, "Mitigation_CO2"] = co2

        # Stability
        climate_cv = (row["Temp_CV"] + row["Precip_CV"]) / 2
        stability = 1 / (1 + climate_cv)

        df.at[idx, "Adaptation_Stability"] = stability

        # Economics
        econ = CONFIG["economic"]
        pulse_yield = econ["pulse_yield_t_ha"] * CPI

        carbon_rev = co2 * econ["carbon_price_usd"]
        pulse_income = area * pulse_yield * econ["pulse_price_usd_t"]
        seed_cost = area * econ["seed_cost_usd_ha"]

        net = carbon_rev + pulse_income - seed_cost

        df.at[idx, "Net_Annual_Benefit"] = net
        df.at[idx, "Net_Benefit_per_Ha"] = net / area

    mask = df["compute_flag"]

    # ---------------- CSA Index ----------------

    mit = df.loc[mask, "Mitigation_CO2"]
    stab = df.loc[mask, "Adaptation_Stability"]

    mit_norm = (mit - mit.min()) / (mit.max() - mit.min())
    stab_norm = (stab - stab.min()) / (stab.max() - stab.min())

    mit_norm = mit_norm.clip(lower=1e-6)
    stab_norm = stab_norm.clip(lower=1e-6)

    w_m = CONFIG["csa_weights"]["mitigation"]
    w_a = CONFIG["csa_weights"]["adaptation"]

    df.loc[mask, "CSA_Index"] = (
        (mit_norm ** w_m) * (stab_norm ** w_a)
    )

    # ---------------- Priority Index ----------------

    area_vals = df.loc[mask, "Rice_Fallows"]
    area_norm = (area_vals - area_vals.min()) / \
                (area_vals.max() - area_vals.min())
    area_norm = area_norm.clip(lower=1e-6)

    df.loc[mask, "Area_Norm"] = area_norm

    w_p = CONFIG["priority_weights"]["performance"]
    w_area = CONFIG["priority_weights"]["area"]

    df.loc[mask, "Priority_Index"] = (
        (df.loc[mask, "CSA_Index"] ** w_p) *
        (area_norm ** w_area)
    )

    # ---------------- Quadrant Classification ----------------

    csa_med = df.loc[mask, "CSA_Index"].median()
    area_med = df.loc[mask, "Area_Norm"].median()

    def classify(row):
        if not row["compute_flag"]:
            return "Not Eligible"
        if row["CSA_Index"] >= csa_med and row["Area_Norm"] >= area_med:
            return "High Performance – High Scale"
        if row["CSA_Index"] >= csa_med:
            return "High Performance – Low Scale"
        if row["Area_Norm"] >= area_med:
            return "Low Performance – High Scale"
        return "Low Performance – Low Scale"

    df["Priority_Class"] = df.apply(classify, axis=1)

    # Save results
    df.to_csv(output_dir / "CSA_Final_Results.csv", index=False)

    # Generate figures
    generate_figures(df, gdf, output_dir)

    print("✅ Complete CSA + Priority + Quadrant analysis finished.")

if __name__ == "__main__":
    main()

✅ All 8 figures saved to csa_india_final
✅ Complete CSA + Priority + Quadrant analysis finished.
